In [51]:
from PIL import Image
import glob
import numpy as np
import time
import h5py
wd_frames = '/home/emsala/Documenten/Studie/These/phashing/dataset/films/film_1/frames/*'
frame_paths = sorted(glob.glob(wd_frames))

In [45]:
def check_conversion_effect(frame_path):
    pil_image_original = Image.open(frame_path).convert("RGB") 
    pixels_original = np.asarray(pil_image_original)
    image_changed = Image.fromarray(pixels_original)
    pixels_changed = np.asarray(image_changed)
    diff = pixels_changed == pixels_original
    same = np.size(diff) == sum(sum(sum(diff)))
    if same:
        print("no conversion effect")

check_conversion_effect(frame_paths[500])

no conversion effect


In [81]:
def pil_to_np(im):
    pixels = np.asarray(im)
    return pixels

def np_to_pil(pixels):
    im = Image.fromarray(pixels)
    return im

In [47]:
def numpy_concatenation(frame_paths):
    im = Image.open(frame_paths[0]).convert("RGB") 
    pixels = pil_to_np(im)
    images_shape = (pixels.shape[0], pixels.shape[1], pixels.shape[2], 0)
    images = np.zeros(images_shape)

    for frame_path in frame_paths:
        im = Image.open(frame_path).convert("RGB") 
        pixels = pil_to_np(im)
        pixels = np.expand_dims(pixels, axis =3)
        images = np.concatenate((images, pixels), axis = 3)

def efficient_concatenation(frame_paths):
    im = Image.open(frame_paths[0]).convert("RGB") 
    pixels = pil_to_np(im)
    n_frames = len(frame_paths)
    images_shape = (pixels.shape[0], pixels.shape[1], pixels.shape[2], n_frames)
    images = np.zeros(images_shape)

    for i in range(0, n_frames):
        frame_path = frame_paths[i]
        im = Image.open(frame_path).convert("RGB") 
        pixels = pil_to_np(im)
        
        images[:,:,:,i] = pixels
    return images

In [44]:
#test concatenations
start_time = time.time()
images = numpy_concatenation(frame_paths[0:30])
end_time = time.time()
print('time with np.concat: ', end_time-start_time)

start_time = time.time()
images = efficient_concatenation(frame_paths[0:30])
end_time = time.time()
print('time without np.concat: ', end_time-start_time)


time with np.concat:  11.749693632125854
time without np.concat:  3.1731889247894287


In [107]:
#npy
# 900 mb for 20 arrays of size (1080, 1920, 3)
# = 50 mb per array
# vs 500 kb per jpg image
# npy is 1000 times as big as jpg!

# 1000 mb for 30 arrays, dus niet linear increase of memory per array

# kan alleen niet gebruiken want np.concatenate is te langzaam => check h5py

def save_to_npy(frame_paths, batchsize = 50):
    n_images = len(frame_paths)
    
    for i in range(batchsize, n_images, batchsize):
        start = i - batchsize
        end = i
        images = efficient_concatenation(frame_paths[start:end])

        n_images = images.shape[3]
        frame_nr = frame_paths[end].split('/')[-1].split('frame')[1].split('.')[0]
        np.save('images_{}'.format(frame_nr), images)

In [109]:
#npy
# 40 mb for 50 arrays of size (1080, 1920, 3) en 350 MB voor 220 arrays
# = 1 tot 2 mb per array
# vs 500 kb per jpg image

# langzamer opslaan dan numpy maar sneller laden 

def save_to_h5py(frame_paths, batchsize = 20):
    counter = 0
    im = Image.open(frame_paths[0]).convert("RGB") 
    pixels = pil_to_np(im)

    n_images = len(frame_paths)
    
    hdf5_store = h5py.File("./cache.hdf5", "a")
    
    for i in range(batchsize, n_images, batchsize):
        counter += 1

        start = i - batchsize
        end = i
               
        frame_paths_batch = frame_paths[start:end]
        images_shape = (pixels.shape[0], pixels.shape[1], pixels.shape[2], batchsize)
        #print(images_shape)
        images = np.zeros(images_shape)

        for j in range(0, batchsize):
            frame_path = frame_paths_batch[j]
            im = Image.open(frame_path).convert("RGB") 
            pixels = pil_to_np(im)

            images[:,:,:,j] = pixels
        
        hdf5_store.create_dataset("images_{}".format(end), data = images, compression="gzip")
#         if counter == 3:
#             break

True